In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings
import csv
import json
import flask_cors, flask
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from diffusers import AutoPipelineForText2Image
import torch
import base64

warnings.filterwarnings('ignore')

c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.empty_cache()
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sd-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  9.66it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.24.0",
  "_name_or_path": "stabilityai/sd-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
app = Flask(__name__)
CORS(app)

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def insert(self, key, value):
        new_node = Node(key, value)
        new_node.next = self.head
        self.head = new_node

    def search(self, key):
        current = self.head
        while current:
            if current.key == key:
                return current.value
            current = current.next
        return None
    
    def display(self):
        values = []
        current = self.head
        while current:
            values.append((current.key, current.value))
            current = current.next
        return values

class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value): 
        index = self.hash_function(key)
        if self.table[index] is None:
            self.table[index] = LinkedList()
        self.table[index].insert(key, value)

    def search(self, key):
        index = self.hash_function(key)
        if self.table[index] is not None:
            return self.table[index].search(key)
        return None
    
    def display(self):
        table_values = []
        for i, linked_list in enumerate(self.table):
            if linked_list is not None:
                values = linked_list.display()
                for key, value in values:
                    table_values.append((i, key, value))
        return table_values
    
def build_forward_index(data):
    forward_index = HashTable(size=100)
    for article_id, title_words in data["title"].items(): 
        if article_id in data["content"]:
            content_words = data["content"][article_id] 
            words = title_words + content_words

            forward_index.insert(article_id, words)

    return forward_index 
 

def build_inverted_index(data):
    inverted_index = HashTable(size=100)
    for article_id, words in data["content"].items(): 
        topic = data["title"][article_id]
        url = data["url"][article_id]
    
        for word in words:
            if inverted_index.search(word):
                inverted_index.search(word).append({"article_id": article_id, "title": topic, "url": url})
            else:
                inverted_index.insert(word, [{"article_id": article_id, "title": topic, "url": url}])

    return inverted_index

def single_word_search(inverted_index, word):
    return inverted_index.search(word)

def multi_word_search_2(inverted_index, query):
    result = set()
    words = query.split()
    if words:
        result = set(inverted_index.search(words[0]))
        for word in words[1:]:
            result.intersection_update(inverted_index.search(word))
    return list(result)

def multi_word_search(inverted_index, query):
    result = []
    words = query.split()

    if words:
        result = inverted_index.search(words[0]) if inverted_index.search(words[0]) else []
        for word in words[1:]:
            current_result = inverted_index.search(word)
            if current_result:
                result = [d for d in result if d in current_result]

    return result


from collections import Counter

def rank_results(input_list): 
    counts = Counter(tuple(item['article_id']) for item in input_list) 
    sorted_list = sorted(input_list, key=lambda item: counts[tuple(item['article_id'])], reverse=True) 
    unique_set = set()
    output = [item for item in sorted_list if tuple(item['title']) not in unique_set and not unique_set.add(tuple(item['article_id']))]
    
    return output

def display_results(results):
    for article_id in results:
        print(f"Article {article_id}: {data['content'][article_id]}")

def add_content(data, new_article):
    article_id = str(len(data["index"]))
    data["index"][article_id] = len(data["index"])
    data["source"][article_id] = new_article[0]
    data["title"][article_id] = new_article[1]
    data["content"][article_id] = new_article[2]

    return data

def remove_duplicates(input_list, key = None):
    seen = set()
    result = []
    
    for item in input_list: 
        hashable_item = frozenset(item.items()) if key is None else item.get(key)
        
        if hashable_item not in seen:
            seen.add(hashable_item)
            result.append(item)
    
    return result

In [4]:
def load_data_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

json_file_path = "Files\cleaned.json"
json_data = load_data_from_json(json_file_path)

forward_index = build_forward_index(json_data)
inverted_index = build_inverted_index(json_data)

In [5]:
@app.route("/search_1", methods=["GET"], endpoint='single_word_search')
def single_word_search():
    word = request.args.get('word')
    ranked_results = rank_results(remove_duplicates(inverted_index.search(word), key="article_id"))

    article_ids = [result['article_id'] for result in ranked_results]
    titles = [result['title'] for result in ranked_results]
    urls = [result['url'] for result in ranked_results] 
    json_response = jsonify(article_ids=article_ids, titles=titles, urls=urls)

    return json_response

@app.route("/search_2", methods=["GET"], endpoint='multi_word_search')
def multi_word_search(): 
    query = request.args.get('word')
    result = []
    words = query.split()

    if words:
        result = inverted_index.search(words[0]) if inverted_index.search(words[0]) else []
        for word in words[1:]:
            current_result = inverted_index.search(word)
            if current_result:
                result = [d for d in result if d in current_result]

    ranked_results = rank_results(remove_duplicates(result, key="article_id"))

    article_ids = [result['article_id'] for result in ranked_results]
    titles = [result['title'] for result in ranked_results]
    urls = [result['url'] for result in ranked_results]

    json_response = jsonify(article_ids=article_ids, titles=titles, urls=urls)

    return json_response 


@app.route("/gen", methods=["GET"], endpoint='genai_tool')
def genai_tool():
    word = request.args.get('word') 
    image = pipe(prompt=word, num_inference_steps=1, guidance_scale=0.0).images[0]

    # Save the image to the "Photos" folder
    folder_path = os.path.join(os.getcwd(), "frontend-gui/public/Photos")
    os.makedirs(folder_path, exist_ok=True)
 
    file_name = "image"

    file_path = os.path.join(folder_path, f"{file_name}.jpeg")  # You can change the extension to 'png' if needed
    image.save(file_path)

    # Convert the image to base64
    image_base64 = base64.b64encode(image.tobytes()).decode('utf-8')


    # Create a JSON response with the base64-encoded image
    json_response = {
        'word': word,
        'image': image_base64
    }

    return jsonify(json_response)


@app.route("/add", methods=["GET"])
def add_content(data, new_article):
    article_id = str(len(data["index"]))
    data["index"][article_id] = len(data["index"])
    data["source"][article_id] = new_article[0]
    data["title"][article_id] = new_article[1]
    data["content"][article_id] = new_article[2]

    return data

if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
100%|██████████| 1/1 [00:03<00:00,  3.04s/it]
127.0.0.1 - - [09/Dec/2023 00:30:10] "GET /gen?word=business HTTP/1.1" 200 -
100%|██████████| 1/1 [00:00<00:00,  3.70it/s]
127.0.0.1 - - [09/Dec/2023 00:30:39] "GET /gen?word=Study HTTP/1.1" 200 -
[2023-12-09 00:30:54,350] ERROR in app: Exception on /search_1 [GET]
Traceback (most recent call last):
  File "c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
             